<a href="https://colab.research.google.com/github/Hubajube/colab/blob/main/Ken's_VQGAN%2BCLIP_(Zooming)_(z%2Bquantize_method_with_addons%2Bimage_manipulation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate images from text phrases with VQGAN and CLIP (z + quantize method), with animation and keyframes

Notebook by Katherine Crowson (https://github.com/crowsonkb, https://twitter.com/RiversHaveWings). The original BigGAN + CLIP method was made by https://twitter.com/advadnoun. Translated into Spanish and added explanations, and modifications by Eleiber#8347, and the friendly interface was made thanks to Abulafia#3734. Translated back into English, and zoom, pan, rotation, and keyframes features by Chigozie Nri (https://github.com/chigozienri, https://twitter.com/chigozienri)
If you encounter problems using it, you are welcome to ask me to fix it at https://twitter.com/chigozienri

Slight modifications by : Justin John

Modifications by Ken Boone:
1) Edit key frames using Google Sheets.
2) Allow for off-center rotation/zooming.
3) Allow for tilt (perspective warping).
4) Frames now have 5 digits instead of 4.
5) Can restart at a selected frame.


*ToDo: Add more models*

# How to use this notebook

This is an example of a Jupyter Notebook, running in Google Colab

It runs Python code in your browser. It's not hard to use, even if you haven't run code before.

First, in the menu bar, click Runtime>Change Runtime Type, and ensure that under "Hardware Accelerator" it says "GPU". If not, choose "GPU" from the drop-down menu, and click Save.

Then, run each of the cells in the notebook, one by one. Make sure to run all of them in order! Click in the cell, and press Shift-Enter on your keyboard. This will run the code in the cell, and then move to the next cell.

Follow the instructions in each cell, and you'll have an AI image in no time!

# Load Google Drive

Long-running colab notebooks might halt, and discard all progress. For this reason, it's useful (although optional) to save the images as they are produced in your personal google drive. Run the cell below to load google drive, click the link, sign in, paste the code generated into the prompt, and press enter.

In [ ]:
#@markdown #**Log into Google** 


from google.colab import auth
auth.authenticate_user()


import gspread
from gspread import utils
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())


from google.colab import drive
drive.mount('/content/gdrive')

working_dir = '/content/gdrive/MyDrive/vqgan'




Mounted at /content/gdrive


If you choose not to use google drive, uncomment the cell below and run it instead.

In [ ]:
#@markdown **Double-click here and uncomment this code if you don't want to use Google Drive**
# working_dir = '/content'

In [ ]:
# @title **Licensed under the MIT License**

# Copyright (c) 2021 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

In [ ]:
#@markdown #**Check GPU type**
#@markdown ### Factory reset runtime if you don't have the desired GPU.

#@markdown ---




#@markdown V100 = Excellent (*Available only for Colab Pro users*)

#@markdown P100 = Very Good (*Available only for Colab Pro users*)

#@markdown T4 = Good (*Available only for Colab Pro users*)

#@markdown K80 = (*Untested*)

#@markdown P4 = (*Not Recommended*) 

#@markdown ---

!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-08c0a333-37e8-2576-a379-ff6822bee4f4)


In [ ]:
# @title #**Library installation**
# @markdown This cell will take a while because you have to download multiple libraries

print("Downloading CLIP...")
!git clone https://github.com/openai/CLIP                 &> /dev/null
 
print("Downloading Python AI libraries...")
!git clone https://github.com/CompVis/taming-transformers &> /dev/null
!pip install ftfy regex tqdm omegaconf pytorch-lightning  &> /dev/null
!pip install kornia                                       &> /dev/null
!pip install einops                                       &> /dev/null
 
print("Installing libraries for handling metadata...")
!pip install stegano                                      &> /dev/null
!apt install exempi                                       &> /dev/null
!pip install python-xmp-toolkit                           &> /dev/null
!pip install imgtag                                       &> /dev/null
!pip install pillow==7.1.2                                &> /dev/null
 
print("Installing Python video creation libraries...")
!pip install imageio-ffmpeg &> /dev/null
path = f'{working_dir}/steps'
!mkdir --parents {path}
print("Installation finished.")

Installing libraries for handling metadata...
Installing Python video creation libraries...
Installation finished.


In [ ]:
#@title #**Selection of models to download**
#@markdown By default, the notebook downloads Model 16384 from ImageNet. There are others such as ImageNet 1024, COCO-Stuff, WikiArt 1024, WikiArt 16384, FacesHQ or S-FLCKR, which are not downloaded by default, since it would be in vain if you are not going to use them, so if you want to use them, simply select the models to download.

imagenet_1024 = False #@param {type:"boolean"}
imagenet_16384 = True #@param {type:"boolean"}
coco = False #@param {type:"boolean"}
faceshq = False #@param {type:"boolean"}
wikiart_16384 = False #@param {type:"boolean"}
sflckr = False #@param {type:"boolean"}

if imagenet_1024:
  !curl -L -o vqgan_imagenet_f16_1024.yaml -C - 'https://heibox.uni-heidelberg.de/d/8088892a516d4e3baf92/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1' #ImageNet 1024
  !curl -L -o vqgan_imagenet_f16_1024.ckpt -C - 'https://heibox.uni-heidelberg.de/d/8088892a516d4e3baf92/files/?p=%2Fckpts%2Flast.ckpt&dl=1'  #ImageNet 1024
if imagenet_16384:
  !curl -L -o vqgan_imagenet_f16_16384.yaml -C - 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1' #ImageNet 16384
  !curl -L -o vqgan_imagenet_f16_16384.ckpt -C - 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fckpts%2Flast.ckpt&dl=1' #ImageNet 16384
if coco:
  !curl -L -o coco.yaml -C - 'https://dl.nmkd.de/ai/clip/coco/coco.yaml' #COCO
  !curl -L -o coco.ckpt -C - 'https://dl.nmkd.de/ai/clip/coco/coco.ckpt' #COCO
if faceshq:
  !curl -L -o faceshq.yaml -C - 'https://drive.google.com/uc?export=download&id=1fHwGx_hnBtC8nsq7hesJvs-Klv-P0gzT' #FacesHQ
  !curl -L -o faceshq.ckpt -C - 'https://app.koofr.net/content/links/a04deec9-0c59-4673-8b37-3d696fe63a5d/files/get/last.ckpt?path=%2F2020-11-13T21-41-45_faceshq_transformer%2Fcheckpoints%2Flast.ckpt' #FacesHQ
if wikiart_16384:
  !curl -L -o wikiart_16384.yaml -C - 'http://eaidata.bmk.sh/data/Wikiart_16384/wikiart_f16_16384_8145600.yaml' #WikiArt 16384
  !curl -L -o wikiart_16384.ckpt -C - 'http://eaidata.bmk.sh/data/Wikiart_16384/wikiart_f16_16384_8145600.ckpt' #WikiArt 16384
if sflckr:
  !curl -L -o sflckr.yaml -C - 'https://heibox.uni-heidelberg.de/d/73487ab6e5314cb5adba/files/?p=%2Fconfigs%2F2020-11-09T13-31-51-project.yaml&dl=1' #S-FLCKR
  !curl -L -o sflckr.ckpt -C - 'https://heibox.uni-heidelberg.de/d/73487ab6e5314cb5adba/files/?p=%2Fcheckpoints%2Flast.ckpt&dl=1' #S-FLCKR

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   692  100   692    0     0    641      0  0:00:01  0:00:01 --:--:--  112k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  934M  100  934M    0     0  10.7M      0  0:01:27  0:01:27 --:--:-- 11.4M


In [ ]:
# @title #**Loading of libraries and definitions**
 
import argparse
import math
from pathlib import Path
import sys
import os
import cv2
import pandas as pd
import numpy as np
import subprocess
 
sys.path.append('./taming-transformers')

# Some models include transformers, others need explicit pip install
try:
    import transformers
except Exception:
    !pip install transformers
    import transformers

from IPython import display
from base64 import b64encode
from omegaconf import OmegaConf
from PIL import Image, ImageFont, ImageDraw
from taming.models import cond_transformer, vqgan
import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms import functional as TF
from tqdm.notebook import tqdm
 
from CLIP import clip
import kornia.augmentation as K
import numpy as np
import imageio
from PIL import ImageFile, Image
#from imgtag import ImgTag    # metadata 
#from libxmp import *         # metadata
#import libxmp                # metadata
from stegano import lsb
import json
from datetime import datetime

ImageFile.LOAD_TRUNCATED_IMAGES = True
 
def sinc(x):
    return torch.where(x != 0, torch.sin(math.pi * x) / (math.pi * x), x.new_ones([]))
 
 
def lanczos(x, a):
    cond = torch.logical_and(-a < x, x < a)
    out = torch.where(cond, sinc(x) * sinc(x/a), x.new_zeros([]))
    return out / out.sum()
 
 
def ramp(ratio, width):
    n = math.ceil(width / ratio + 1)
    out = torch.empty([n])
    cur = 0
    for i in range(out.shape[0]):
        out[i] = cur
        cur += ratio
    return torch.cat([-out[1:].flip([0]), out])[1:-1]
 
 
def resample(input, size, align_corners=True):
    n, c, h, w = input.shape
    dh, dw = size
 
    input = input.view([n * c, 1, h, w])
 
    if dh < h:
        kernel_h = lanczos(ramp(dh / h, 2), 2).to(input.device, input.dtype)
        pad_h = (kernel_h.shape[0] - 1) // 2
        input = F.pad(input, (0, 0, pad_h, pad_h), 'reflect')
        input = F.conv2d(input, kernel_h[None, None, :, None])
 
    if dw < w:
        kernel_w = lanczos(ramp(dw / w, 2), 2).to(input.device, input.dtype)
        pad_w = (kernel_w.shape[0] - 1) // 2
        input = F.pad(input, (pad_w, pad_w, 0, 0), 'reflect')
        input = F.conv2d(input, kernel_w[None, None, None, :])
 
    input = input.view([n, c, h, w])
    return F.interpolate(input, size, mode='bicubic', align_corners=align_corners)
 
 
class ReplaceGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x_forward, x_backward):
        ctx.shape = x_backward.shape
        return x_forward
 
    @staticmethod
    def backward(ctx, grad_in):
        return None, grad_in.sum_to_size(ctx.shape)
 
 
replace_grad = ReplaceGrad.apply
 
 
class ClampWithGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, min, max):
        ctx.min = min
        ctx.max = max
        ctx.save_for_backward(input)
        return input.clamp(min, max)
 
    @staticmethod
    def backward(ctx, grad_in):
        input, = ctx.saved_tensors
        return grad_in * (grad_in * (input - input.clamp(ctx.min, ctx.max)) >= 0), None, None
 
 
clamp_with_grad = ClampWithGrad.apply
 
 
def vector_quantize(x, codebook):
    d = x.pow(2).sum(dim=-1, keepdim=True) + codebook.pow(2).sum(dim=1) - 2 * x @ codebook.T
    indices = d.argmin(-1)
    x_q = F.one_hot(indices, codebook.shape[0]).to(d.dtype) @ codebook
    return replace_grad(x_q, x)
 
 
class Prompt(nn.Module):
    def __init__(self, embed, weight=1., stop=float('-inf')):
        super().__init__()
        self.register_buffer('embed', embed)
        self.register_buffer('weight', torch.as_tensor(weight))
        self.register_buffer('stop', torch.as_tensor(stop))
 
    def forward(self, input):
        input_normed = F.normalize(input.unsqueeze(1), dim=2)
        embed_normed = F.normalize(self.embed.unsqueeze(0), dim=2)
        dists = input_normed.sub(embed_normed).norm(dim=2).div(2).arcsin().pow(2).mul(2)
        dists = dists * self.weight.sign()
        return self.weight.abs() * replace_grad(dists, torch.maximum(dists, self.stop)).mean()
 
 
def parse_prompt(prompt):
    vals = prompt.rsplit(':', 2)
    vals = vals + ['', '1', '-inf'][len(vals):]
    return vals[0], float(vals[1]), float(vals[2])
 
 
class MakeCutouts(nn.Module):
    def __init__(self, cut_size, cutn, cut_pow=1.):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.cut_pow = cut_pow
        self.augs = nn.Sequential(
            K.RandomHorizontalFlip(p=0.5),
            # K.RandomSolarize(0.01, 0.01, p=0.7),
            K.RandomSharpness(0.3,p=0.4),
            K.RandomAffine(degrees=30, translate=0.1, p=0.8, padding_mode='border'),
            K.RandomPerspective(0.2,p=0.4),
            K.ColorJitter(hue=0.01, saturation=0.01, p=0.7))
        self.noise_fac = 0.1
 
 
    def forward(self, input):
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)
        min_size = min(sideX, sideY, self.cut_size)
        cutouts = []
        for _ in range(self.cutn):
            size = int(torch.rand([])**self.cut_pow * (max_size - min_size) + min_size)
            offsetx = torch.randint(0, sideX - size + 1, ())
            offsety = torch.randint(0, sideY - size + 1, ())
            cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
            cutouts.append(resample(cutout, (self.cut_size, self.cut_size)))
        batch = self.augs(torch.cat(cutouts, dim=0))
        if self.noise_fac:
            facs = batch.new_empty([self.cutn, 1, 1, 1]).uniform_(0, self.noise_fac)
            batch = batch + facs * torch.randn_like(batch)
        return batch
 
 
def load_vqgan_model(config_path, checkpoint_path):
    config = OmegaConf.load(config_path)
    if config.model.target == 'taming.models.vqgan.VQModel':
        model = vqgan.VQModel(**config.model.params)
        model.eval().requires_grad_(False)
        model.init_from_ckpt(checkpoint_path)
    elif config.model.target == 'taming.models.cond_transformer.Net2NetTransformer':
        parent_model = cond_transformer.Net2NetTransformer(**config.model.params)
        parent_model.eval().requires_grad_(False)
        parent_model.init_from_ckpt(checkpoint_path)
        model = parent_model.first_stage_model
    else:
        raise ValueError(f'unknown model type: {config.model.target}')
    del model.loss
    return model
 
 
def resize_image(image, out_size):
    ratio = image.size[0] / image.size[1]
    area = min(image.size[0] * image.size[1], out_size[0] * out_size[1])
    size = round((area * ratio)**0.5), round((area / ratio)**0.5)
    return image.resize(size, Image.LANCZOS)


def read_image_workaround(path):
    """OpenCV reads images as BGR, Pillow saves them as RGB. Work around
    this incompatibility to avoid colour inversions."""
    print (path)
    im_tmp = cv2.imread(path)
    return cv2.cvtColor(im_tmp, cv2.COLOR_BGR2RGB)

#prime the pump    
img = read_image_workaround(f'{working_dir}/prime.png')

     |████████████████████████████████| 3.5 MB 4.1 MB/s 
     |████████████████████████████████| 67 kB 6.3 MB/s 
     |████████████████████████████████| 6.8 MB 68.1 MB/s 
     |████████████████████████████████| 895 kB 63.4 MB/s 
/content/gdrive/MyDrive/vqgan/prime.png


## **Instructions for setting parameters:**

| Parameter  |  Usage |
|---|---|
| `key_frames` | Whether to use key frames to change the parameters over the course of the run |
|  `text_prompts` |  Text prompts, separated by "\|" |
| `width` | Width of the output, in pixels. This will be rounded down to a multiple of 16 |
| `height` | Height of the output, in pixels. This will be rounded down to a multiple of 16 |
| `model` | Choice of model, must be downloaded above |
| `interval` | How often to display the frame in the notebook (doesn't affect the actual output) |
| `initial_image` | Image to start with (relative path to file) |
| `target_images` | Image prompts to target, separated by "|" (relative path to files) |
| `seed` | Random seed, if set to a positive integer the run will be repeatable (get the same output for the same input each time, if set to -1 a random seed will be used. |
| `max_frames` | Number of frames for the animation |
| `angle` | Angle in degrees to rotate clockwise between each frame |
| `zoom` | Factor to zoom in each frame, 1 is no zoom, less than 1 is zoom out, more than 1 is zoom in (negative is uninteresting, just adds an extra 180 rotation beyond that in angle) |
| `translation_x` | Number of pixels to shift right each frame |
| `translation_y` | Number of pixels to shift down each frame |
| `center_offset_y` | 0; -1 is far left side; 1 is far right side |
| `center_offset_y` | 0; -1 is far top side; 1 is far bottom side |
| `vtilt` | 0; 1 tilts back on the top; -1 tilts back on the bottom |
| `htilt` | 0; 1 tilts back on the left; -1 tilts back on the right |
| `custom1` | custom parameter handled in code |
| `custom2` | custom parameter handled in code |
| `custom3` | custom parameter handled in code |
| `custom4` | custom parameter handled in code |
| `custom5` | custom parameter handled in code |
| `iterations_per_frame` | Number of times to run the VQGAN+CLIP method each frame |
| `save_all_iterations` | Debugging, set False in normal operation |

---------

Transformations (zoom, rotation, and translation)

On each frame, the network restarts, is fed a version of the output zoomed in by `zoom` as the initial image, rotated clockwise by `angle` degrees, translated horizontally by `translation_x` pixels, and translated vertically by `translation_y` pixels. Then it runs `iterations_per_frame` iterations of the VQGAN+CLIP method. 0 `iterations_per_frame` is supported, to help test out the transformations without changing the image.

For `iterations_per_frame = 1` (recommended for more abstract effects), the resulting images will not have much to do with the prompts, but at least one prompt is still required.

In normal use, only the last iteration of each frame will be saved, but for trouble-shooting you can set `save_all_iterations` to True, and every iteration of each frame will be saved.

----------------

Mainly what you will have to modify will be `text_prompts`: there you can place the prompt(s) you want to generate (separated with |). It is a list because you can put more than one text, and so the AI tries to 'mix' the images, giving the same priority to both texts. You can also assign weights, to bias the priority towards one prompt or another, or negative weights, to remove an element (for example, a colour).

Example of weights with decimals:

Text : rubber:0.5 | rainbow:0.5

To use an initial image to the model, you just have to upload a file to the Colab environment (in the section on the left), and then modify `initial_image`: putting the exact name of the file. Example: sample.png

You can also change the model by changing the line that says `model`. Currently 1024, 16384, WikiArt, S-FLCKR and COCO-Stuff are available. To activate them you have to have downloaded them first, and then you can simply select it.

You can also use `target_images`, which is basically putting one or more images on it that the AI will take as a "target", fulfilling the same function as putting text on it. To put more than one you have to use | as a separator.

------------

Key Frames

If `key_frames` is set to True, you are able to change the parameters over the course of the run.
To do this, put the parameters in in the following format:
10:(0.5), 20: (1.0), 35: (-1.0)

This means at frame 10, the value should be 0.5, at frame 20 the value should be 1.0, and at frame 35 the value should be -1.0. The value at each other frame will be linearly interpolated (that is, before frame 10, the value will be 0.5, between frame 10 and 20 the value will increase frame-by-frame from 0.5 to 1.0, between frame 20 and 35 the value will decrease frame-by-frame from 1.0 to -1.0, and after frame 35 the value will be -1.0)

This also works for text_prompts, e.g. 10:(Apple: 1| Orange: 0), 20: (Apple: 0| Orange: 1| Peach: 1)
will start with an Apple value of 1, once it hits frame 10 it will start decreasing in in Apple and increasing in Orange until it hits frame 20. Note that Peach will have a value of 1 the whole time.

If `key_frames` is set to True, all of the parameters which can be key-framed must be entered in this format.

In [ ]:
# @title **Parameters**
#@markdown #<a name="parameters"></a>

from datetime import datetime


use_sheets = True #@param {type:"boolean"}

key_frames = True #@param {type:"boolean"}
restart_frame = 0#@param {type:"number"}
max_frames = 200#@param {type:"number"}

width =  400#@param {type:"number"}
height =  400#@param {type:"number"}
model = "vqgan_imagenet_f16_16384" #@param ["vqgan_imagenet_f16_16384", "vqgan_imagenet_f16_1024", "wikiart_16384", "coco", "faceshq", "sflckr"]
interval =  10#@param {type:"number"}
initial_image = ""#@param {type:"string"}
seed = 1#@param {type:"number"}
save_all_iterations = False#@param  {type:"boolean"}

text_prompts = "0:(:6|Favia coral :0),11196:(brain coral:6|Favia coral :2|Stylophora:0),11424:(face:-5|anime:-5|ocean underwater:9|anemone:5|bubbles:4|brain coral:3|Favia coral :6|Stylophora:2|seaweed:3|squid:0),11436:(ocean underwater:8|anemone:2|bubbles:5|brain coral:0|Favia coral :3|Stylophora:6|seaweed:4|squid:3|coral:0),11808:(anemone:0|bubbles:6|Favia coral :0|Stylophora:3|seaweed:6|squid:4|coral:5|seahorse:0|eel:0),11820:(ocean underwater:7|bubbles:11|seaweed:4|squid:2|coral:6|seahorse:7|eel:6|angler fish:0),12094:(bubbles:16|Stylophora:0|seaweed:2|squid:0|coral:4|seahorse:8|eel:7|angler fish:1|blue hole:0),12190:(ocean underwater:6|bubbles:6|Goniopora:0|seaweed:0|coral:2|seahorse:10|eel:5|angler fish:4|blue hole:5),12286:(bubbles:3|Goniopora:2|Fungiidae:0|coral:1|seahorse:12|eel:3|angler fish:4|blue hole:6|vortex:0),12382:(bubbles:0|coral:0|seahorse:7|angler fish:8|blue hole:7|vortex:2),12478:(Goniopora:6|Fungiidae:2|seahorse:5|eel:2|angler fish:9|blue hole:9|vortex:3),12574:(seahorse:2|eel:0|angler fish:3|blue hole:16|vortex:3),12670:(Goniopora:3|Fungiidae:6|seahorse:0|angler fish:2|blue hole:7|vortex:8),12766:(angler fish:0|blue hole:3|vortex:6|Zeta Ophiuchi:0),12862:(ocean underwater:0|Goniopora:0|Fungiidae:8|blue hole:0|vortex:8|Zeta Ophiuchi:5)" #@param {type:"string"}
target_images = ""#@param {type:"string"}
angle = "12766:(0),12862:(5)"#@param {type:"string"}
zoom = "11183:(1.02),11196:(0.95),11424:(0.95),11436:(0.95),11808:(0.99),11820:(0.98),12094:(0.99),12382:(0.99),12478:(1.01),12574:(1.02),12670:(1.03),12766:(1.02),12862:(1.03)"#@param {type:"string"}
translation_x = "11183:(3),11196:(5),11424:(7),11436:(5),11808:(4),11820:(2),12094:(4),12382:(3),12478:(2),12574:(-4),12670:(-2)"#@param {type:"string"}
translation_y = "11183:(2),11196:(0),11424:(-3),11436:(-2),11808:(1),11820:(0),12094:(0),12382:(0),12478:(3),12574:(0),12670:(-2)"#@param {type:"string"}
center_offset_x = "11183:(-0.1),11196:(0.1),11424:(0.3),11436:(0.1),11808:(0.3),11820:(0.3),12094:(0.2),12382:(0.2),12478:(0.2),12574:(-0.3),12670:(-0.2),12862:(-0.4)"#@param {type:"string"}
center_offset_y = "11183:(-0.4),11196:(-0.5),11424:(-0.7),11436:(-0.3),11808:(0),11820:(0.2),12094:(0.2),12382:(0.2),12478:(0.9),12574:(0.5),12670:(-0.2),12862:(-0.5)"#@param {type:"string"}
vtilt = "" #@param {type:"string"}
htilt = "" #@param {type:"string"}
custom1 = "" #@param {type:"string"}
custom2 = "" #@param {type:"string"}
custom3 = "" #@param {type:"string"}
custom4 = "" #@param {type:"string"}
custom5 = "" #@param {type:"string"}
iterations_per_frame = "9397:(15)"#@param {type:"string"}


if initial_image != "":
    print(
        "WARNING: You have specified an initial image. Note that the image resolution "
        "will be inherited from this image, not whatever width and height you specified. "
        "If the initial image resolution is too high, this can result in out of memory errors."
    )
elif width * height > 160000:
    print(
        "WARNING: The width and height you have specified may be too high, in which case "
        "you will encounter out of memory errors either at the image generation stage or the "
        "video synthesis stage. If so, try reducing the resolution"
    )
model_names={
    "vqgan_imagenet_f16_16384": 'ImageNet 16384',
    "vqgan_imagenet_f16_1024":"ImageNet 1024", 
    "wikiart_1024":"WikiArt 1024",
    "wikiart_16384":"WikiArt 16384",
    "coco":"COCO-Stuff",
    "faceshq":"FacesHQ",
    "sflckr":"S-FLCKR"
}
model_name = model_names[model]

if seed == -1:
    seed = None

def clean_worksheet(worksheet):
  framecell = worksheet.find('frame')
  framecellrow = framecell.row
  framecellcol = framecell.col
  ret = worksheet.get_all_values()

  #remove all the initial extra rows
  for row in range(1, framecellrow-1):
    ret.pop(0)
    #print(f'removing row: {ret.pop(0)}')

  #remove all the initial extra cols
  for rownum in range(0, len(ret)):
    for colnum in (0, framecellcol):
      ret[rownum].pop(0)
      #print(f'removing col: {ret[rownum].pop(0)}')

  #remove trailing rows
  rowcount = len(ret)
  for rownum in range(1,rowcount+1):
    if rownum>=rowcount:
      break
    if ret[rownum][0] == "":
      break

  for rowtodelete in range(rownum, rowcount):
    ret.pop(rownum)

  #remove trailing cols
  row = ret[1]
  colcount = len(row)
  for colnum in range(1,colcount+1):
    if colnum>=colcount:
      break
    if row[colnum] == "":
      break

  for coltodelete in range(colnum, colcount):
    for row in ret:
      row.pop(colnum)

  return ret

#def getweights(worksheet, rowstart, rowend, colnum):
#  weights = []
#  for rownum in range(rowstart, rowend+1):
#    weights.append(worksheet.cell(rownum, colnum).value)
#  return weights

def get_frames(allcells):
  ret = []
  for row in allcells[2:]:
    ret.append(row[0])
  return ret

def get_prompts(allcells, prompt_type):
  prompt_dict = {}
  colnum = 1
  for cell in allcells[1][1:]:
    if allcells[0][colnum] == prompt_type:
      frame_dict = {}
      for row in allcells[2:]:
        frame_dict[row[0]] = row[colnum]
      if prompt_type == 'img':
        prompt_name = f'/content/gdrive/MyDrive/vqgan/target/{allcells[1][colnum]}.png'
      else:
        prompt_name = allcells[1][colnum]
      prompt_dict[prompt_name] = frame_dict
    colnum = colnum + 1
  return prompt_dict


def fmt_parameter_weights(frames, parameters, parameter_name):
  fmt = ''
  for f,weights in parameters[parameter_name].items():
    if weights != '':
      fmt = fmt + f'{f}:({weights}),'
  return fmt[:-1]


def fmt_prompt_weights(frames, prompts):
  fmt = ''
  for f in frames:
    fmt = fmt + f'{f}:('
    lenprefix = len(f'{f}:(')
    hasweights = 0
    for p,weights in prompts.items():
      if weights[f] != '':
        fmt = fmt + f'{p}:{weights[f]}|'
        hasweights = 1
    if hasweights == 1:
      fmt = fmt[:-1] + '),'
    else:
      fmt = fmt[:-lenprefix]
  return fmt[:-1]

def parse_key_frames(string, prompt_parser=None):
    import re
    pattern = r'((?P<frame>[0-9]+):[\s]*[\(](?P<param>[\S\s]*?)[\)])'
    frames = dict()
    for match_object in re.finditer(pattern, string):
        frame = int(match_object.groupdict()['frame'])
        param = match_object.groupdict()['param']
        if prompt_parser:
            frames[frame] = prompt_parser(param)
        else:
            frames[frame] = param

    if frames == {} and len(string) != 0:
        raise RuntimeError('Key Frame string not correctly formatted')
    return frames

def get_inbetweens(key_frames, integer=False):
    key_frame_series = pd.Series([np.nan for a in range(max_frames)])
    for i, value in key_frames.items():
        key_frame_series[i] = value
    key_frame_series = key_frame_series.astype(float)
    key_frame_series = key_frame_series.interpolate(limit_direction='both')
    if integer:
        return key_frame_series.astype(int)
    return key_frame_series

def split_key_frame_text_prompts(frames):
    prompt_dict = dict()
    for i, parameters in frames.items():
        prompts = parameters.split('|')
        for prompt in prompts:
            string, value = prompt.split(':')
            string = string.strip()
            value = float(value.strip())
            if string in prompt_dict:
                prompt_dict[string][i] = value
            else:
                prompt_dict[string] = {i: value}
    prompt_series_dict = dict()
    for prompt, values in prompt_dict.items():
        value_string = (
            ', '.join([f'{value}: ({values[value]})' for value in values])
        )
        prompt_series = get_inbetweens(parse_key_frames(value_string))
        prompt_series_dict[prompt] = prompt_series
    prompt_list = []
    for i in range(max_frames):
        prompt_list.append(
            ' | '.join(
                [f'{prompt}: {prompt_series_dict[prompt][i]}'
                 for prompt in prompt_series_dict]
            )
        )
    return prompt_list

def writefmt(f,fmtstring):
    for item in fmtstring.split(','):
        f.write("%s\n" % item)
    f.write('\n')

def createlog():
    today = datetime.now().strftime("%Y%m%d%H%M%S")
    filename = f'/content/gdrive/MyDrive/vqgan/logs/vqgan-{today}.log'

    with open(filename, "w") as f:
        f.write(f'text_prompts:\n')
        writefmt(f,text_prompts)
        f.write(f'width:{width}\n')
        f.write(f'height:{height}\n')
        f.write(f'model:{model}\n')
        f.write(f'interval:{interval}\n')
        f.write(f'initial_image:{initial_image}\n')
        f.write(f'target_images:{target_images}\n')
        f.write(f'seed:{seed}\n')
        f.write(f'restart_frame:{restart_frame}\n')
        f.write(f'max_frames:{max_frames}\n')
        f.write('angle:\n')
        writefmt(f,angle)
        f.write('zoom:\n')
        writefmt(f,zoom)
        f.write('translation_x:\n')
        writefmt(f,translation_x)
        f.write('translation_y:\n')
        writefmt(f,translation_y)
        f.write('center_offset_x:\n')
        writefmt(f,center_offset_x)
        f.write('center_offset_y:\n')
        writefmt(f,center_offset_y)
        f.write('htilt:\n')
        writefmt(f,htilt)
        f.write('vtilt:\n')
        writefmt(f,vtilt)
        f.write('custom1:\n')
        writefmt(f,custom1)
        f.write('custom2:\n')
        writefmt(f,custom2)
        f.write('custom3:\n')
        writefmt(f,custom3)
        f.write('custom4:\n')
        writefmt(f,custom4)
        f.write('custom5:\n')
        writefmt(f,custom5)
        f.write('iterations_per_frame:\n')
        writefmt(f,iterations_per_frame)
        f.write('\n')
        f.write(f'{datetime.now()}\n')

    return filename

def updatelog(filename, i):
    with open(filename, "w") as f:
        f.write(f'Frame {i:05d} completed at {datetime.now()}\n')

if use_sheets:
  gc = gspread.authorize(GoogleCredentials.get_application_default())

  spreadsheet = gc.open('VQGan')
  allcells = clean_worksheet(spreadsheet.worksheet("VQGan"))

  #get frames
  frames = get_frames(allcells)

  #get text and image prompts and other keyframed params
  sheets_text_prompts = get_prompts(allcells, 'text')
  sheets_image_prompts = get_prompts(allcells, 'img')
  sheets_parameters = get_prompts(allcells, '')

  key_frames = True
  text_prompts = fmt_prompt_weights(frames, sheets_text_prompts)
  target_images = fmt_prompt_weights(frames, sheets_image_prompts)
  angle = fmt_parameter_weights(frames, sheets_parameters, 'angle')
  zoom = fmt_parameter_weights(frames, sheets_parameters, 'zoom')
  translation_x = fmt_parameter_weights(frames, sheets_parameters, 'translate_x')
  translation_y = fmt_parameter_weights(frames, sheets_parameters, 'translate_y')
  center_offset_x = fmt_parameter_weights(frames, sheets_parameters, 'center_offset_x')
  center_offset_y = fmt_parameter_weights(frames, sheets_parameters, 'center_offset_y')
  iterations_per_frame = fmt_parameter_weights(frames, sheets_parameters, 'iterations')
  vtilt = fmt_parameter_weights(frames, sheets_parameters, 'vtilt')
  htilt = fmt_parameter_weights(frames, sheets_parameters, 'htilt')
  custom1 = fmt_parameter_weights(frames, sheets_parameters, 'custom1')
  custom2 = fmt_parameter_weights(frames, sheets_parameters, 'custom2')
  custom3 = fmt_parameter_weights(frames, sheets_parameters, 'custom3')
  custom4 = fmt_parameter_weights(frames, sheets_parameters, 'custom4')
  custom5 = fmt_parameter_weights(frames, sheets_parameters, 'custom5')
  
if key_frames:
    try:
        text_prompts_series = split_key_frame_text_prompts(
            parse_key_frames(text_prompts)
        )
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `text_prompts` correctly for key frames.\n"
            "Attempting to interpret `text_prompts` as "
            f'"0: ({text_prompts}:1)"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        text_prompts = f"0: ({text_prompts}:1)"
        text_prompts_series = split_key_frame_text_prompts(
            parse_key_frames(text_prompts)
        )

    try:
        target_images_series = split_key_frame_text_prompts(
            parse_key_frames(target_images)
        )
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `target_images` correctly for key frames.\n"
            "Attempting to interpret `target_images` as "
            f'"0: ({target_images}:1)"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        target_images = f"0: ({target_images}:1)"
        target_images_series = split_key_frame_text_prompts(
            parse_key_frames(target_images)
        )

    try:
        angle_series = get_inbetweens(parse_key_frames(angle))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `angle` correctly for key frames.\n"
            "Attempting to interpret `angle` as "
            f'"0: ({angle})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        angle = f"0: ({angle})"
        angle_series = get_inbetweens(parse_key_frames(angle))

    try:
        zoom_series = get_inbetweens(parse_key_frames(zoom))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `zoom` correctly for key frames.\n"
            "Attempting to interpret `zoom` as "
            f'"0: ({zoom})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        zoom = f"0: ({zoom})"
        zoom_series = get_inbetweens(parse_key_frames(zoom))

    try:
        translation_x_series = get_inbetweens(parse_key_frames(translation_x))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `translation_x` correctly for key frames.\n"
            "Attempting to interpret `translation_x` as "
            f'"0: ({translation_x})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        translation_x = f"0: ({translation_x})"
        translation_x_series = get_inbetweens(parse_key_frames(translation_x))

    try:
        translation_y_series = get_inbetweens(parse_key_frames(translation_y))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `translation_y` correctly for key frames.\n"
            "Attempting to interpret `translation_y` as "
            f'"0: ({translation_y})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        translation_y = f"0: ({translation_y})"
        translation_y_series = get_inbetweens(parse_key_frames(translation_y))

    try:
        center_offset_x_series = get_inbetweens(parse_key_frames(center_offset_x))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `center_offset_x` correctly for key frames.\n"
            "Attempting to interpret `center_offset_x` as "
            f'"0: ({center_offset_x})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        center_offset_x = f"0: ({center_offset_x})"
        center_offset_x_series = get_inbetweens(parse_key_frames(center_offset_x))

    try:
        center_offset_y_series = get_inbetweens(parse_key_frames(center_offset_y))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `center_offset_y` correctly for key frames.\n"
            "Attempting to interpret `center_offset_y` as "
            f'"0: ({center_offset_y})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        center_offset_y = f"0: ({center_offset_y})"
        center_offset_y_series = get_inbetweens(parse_key_frames(center_offset_y))

    try:
        vtilt_series = get_inbetweens(parse_key_frames(vtilt))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `vtilt` correctly for key frames.\n"
            "Attempting to interpret `vtilt` as "
            f'"0: ({vtilt})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        vtilt = f"0: ({vtilt})"
        vtilt_series = get_inbetweens(parse_key_frames(vtilt))

    try:
        htilt_series = get_inbetweens(parse_key_frames(htilt))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `htilt` correctly for key frames.\n"
            "Attempting to interpret `htilt` as "
            f'"0: ({htilt})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        htilt = f"0: ({htilt})"
        htilt_series = get_inbetweens(parse_key_frames(htilt))

    try:
        custom1_series = get_inbetweens(parse_key_frames(custom1))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `custom1` correctly for key frames.\n"
            "Attempting to interpret `custom1` as "
            f'"0: ({custom1})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        custom1 = f"0: ({custom1})"
        custom1_series = get_inbetweens(parse_key_frames(custom1))

    try:
        custom2_series = get_inbetweens(parse_key_frames(custom2))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `custom2` correctly for key frames.\n"
            "Attempting to interpret `custom2` as "
            f'"0: ({custom2})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        custom2 = f"0: ({custom2})"
        custom2_series = get_inbetweens(parse_key_frames(custom2))

    try:
        custom3_series = get_inbetweens(parse_key_frames(custom3))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `custom3` correctly for key frames.\n"
            "Attempting to interpret `custom3` as "
            f'"0: ({custom3})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        custom3 = f"0: ({custom3})"
        custom3_series = get_inbetweens(parse_key_frames(custom3))

    try:
        custom4_series = get_inbetweens(parse_key_frames(custom4))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `custom4` correctly for key frames.\n"
            "Attempting to interpret `custom4` as "
            f'"0: ({custom4})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        custom4 = f"0: ({custom4})"
        custom4_series = get_inbetweens(parse_key_frames(custom4))

    try:
        custom5_series = get_inbetweens(parse_key_frames(custom5))
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `custom5` correctly for key frames.\n"
            "Attempting to interpret `custom5` as "
            f'"0: ({custom5})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        custom5 = f"0: ({custom5})"
        custom5_series = get_inbetweens(parse_key_frames(custom5))

    try:
        iterations_per_frame_series = get_inbetweens(
            parse_key_frames(iterations_per_frame), integer=True
        )
    except RuntimeError as e:
        print(
            "WARNING: You have selected to use key frames, but you have not "
            "formatted `iterations_per_frame` correctly for key frames.\n"
            "Attempting to interpret `iterations_per_frame` as "
            f'"0: ({iterations_per_frame})"\n'
            "Please read the instructions to find out how to use key frames "
            "correctly.\n"
        )
        iterations_per_frame = f"0: ({iterations_per_frame})"
        
        iterations_per_frame_series = get_inbetweens(
            parse_key_frames(iterations_per_frame), integer=True
        )
else:
    text_prompts = [phrase.strip() for phrase in text_prompts.split("|")]
    if text_prompts == ['']:
        text_prompts = []
    if target_images == "None" or not target_images:
        target_images = []
    else:
        target_images = target_images.split("|")
        target_images = [image.strip() for image in target_images]

    angle = float(angle)
    zoom = float(zoom)
    translation_x = float(translation_x)
    translation_y = float(translation_y)
    #center_offset_x = float(center_offset_x)
    #center_offset_y = float(center_offset_y)
    iterations_per_frame = int(iterations_per_frame)

logfilename = createlog()

args = argparse.Namespace(
    prompts=text_prompts,
    image_prompts=target_images,
    noise_prompt_seeds=[],
    noise_prompt_weights=[],
    size=[width, height],
    init_weight=0.,
    clip_model='ViT-B/32',
    #ViT-B/16 
    #ViT-L/14
    vqgan_config=f'{model}.yaml',
    vqgan_checkpoint=f'{model}.ckpt',
    step_size=0.1,
    cutn=64,
    cut_pow=1.,
    display_freq=interval,
    seed=seed,
)

In [ ]:
#@markdown **This cell deletes any frames already in the steps directory. If we are restarting, then only files on or after the restart will be deleted.**
deletefiles = False#@param {type:"boolean"} 

path = f'{working_dir}/steps'

if deletefiles:
  if restart_frame==0:
    !rm -r {path}
    !mkdir --parents {path}
  else:
    for i in range(restart_frame, max_frames+1):
      # !/usr/bin/python
      if os.path.exists(f'{path}/{i:04d}.png'):
        print(f'Deleting {path}/{i:04d}.png')
        os.remove(f'{path}/{i:04d}.png')
      if os.path.exists(f'{path}/zoomed_{i:04d}.png'):
        print(f'Deleting {path}/zoomed_{i:04d}.png')
        os.remove(f'{path}/zoomed_{i:04d}.png')


In [ ]:
#@markdown **Delete/clear the generated video**
deletevideo = False#@param {type:"boolean"} 

if deletevideo:
  if key_frames:
      # key frame filename would be too long
      filename = "video.mp4"
  else:
      filename = f"{'_'.join(text_prompts).replace(' ', '')}.mp4"
  filepath = f'{working_dir}/{filename}'

In [ ]:
#@markdown **Transformation functions**

def apply_basic_transformation(img, center_offset, angle, zoom, translation):
  pivot_x = int(np.floor(img.shape[0]*(.5+center_offset[0]/2)))
  pivot_y = int(np.floor(img.shape[1]*(.5+center_offset[1]/2)))
  center = (pivot_x, pivot_y)

  trans_mat = np.float32([[1, 0, translation[0]], [0, 1, translation[1]]])
  rot_mat = cv2.getRotationMatrix2D( center, angle, zoom )

  trans_mat = np.vstack([trans_mat, [0,0,1]])
  rot_mat = np.vstack([rot_mat, [0,0,1]])
  transformation_matrix = np.matmul(rot_mat, trans_mat)

  img = cv2.warpPerspective(
          img,
          transformation_matrix,
          (img.shape[1], img.shape[0]),
          borderMode=cv2.BORDER_WRAP
        )

  return img

def apply_vtilt(img, tilt_amt):    
    top_pinch = int(25*tilt_amt)
    top_dip = int(25*tilt_amt)
    bottom_pinch = -int(25*tilt_amt)
    bottom_dip = 0

    input_pts = np.float32([[0, 0], [width, 0], [width, height], [0, height]])
    output_pts = np.float32([
                            [top_pinch, top_dip], 
                            [width-top_pinch, top_dip], 
                            [width-bottom_pinch, height-bottom_dip], 
                            [bottom_pinch, height-bottom_dip]])

    print(f'vtilt pts:{output_pts}')

    transform_mat = cv2.getPerspectiveTransform(input_pts, output_pts)
    M = cv2.getPerspectiveTransform(input_pts,output_pts)
    return cv2.warpPerspective(
          img,
          M,
          (width, height),
          flags=cv2.INTER_LINEAR,
          borderMode=cv2.BORDER_WRAP)


def apply_htilt(img, tilt_amt):    
    left_pinch = int(25*tilt_amt)
    left_dip = int(25*tilt_amt)
    right_pinch = -int(25*tilt_amt)
    right_dip = 0

    input_pts = np.float32([[0, 0], [width, 0], [width, height], [0, height]])
    output_pts = np.float32([
                         [left_dip, left_pinch], 
                         [width-right_dip, right_pinch], 
                         [width-right_dip, height-right_pinch], 
                         [left_dip, height-left_pinch]])

    transform_mat = cv2.getPerspectiveTransform(input_pts, output_pts)
    M = cv2.getPerspectiveTransform(input_pts,output_pts)
    return cv2.warpPerspective(
          img,
          M,
          (width, height),
          flags=cv2.INTER_LINEAR,
          borderMode=cv2.BORDER_WRAP)

def apply_windowshades(img, stripe_count, translation_x, phase):
  img_mask = np.zeros((width,height,3), np.uint8)

  stripe_height=height/stripe_count
  for stripe in range(0, math.ceil(stripe_count/2)):
    stripe_top=round(stripe*stripe_height*2 + phase*stripe_height)
    stripe_bottom=round(stripe_top+stripe_height)
    #print(stripe_top,stripe_bottom)
    img_mask[stripe_top:stripe_bottom,:] = (255,255,255)

  trans_mat = np.float32(
      [[1, 0, translation_x],
      [0, 1, 0]]
  )

  rot_mat = cv2.getRotationMatrix2D( (0,0), 0, 1 )
  trans_mat = np.vstack([trans_mat, [0,0,1]])
  rot_mat = np.vstack([rot_mat, [0,0,1]])
  transformation_matrix = np.matmul(rot_mat, trans_mat)

  img_1 = cv2.warpPerspective(
      img,
      transformation_matrix,
      (img.shape[1], img.shape[0]),
      borderMode=cv2.BORDER_WRAP
  )

  trans_mat = np.float32(
      [[1, 0, -translation_x],
      [0, 1, translation_y]]
  )
  trans_mat = np.vstack([trans_mat, [0,0,1]])
  transformation_matrix = np.matmul(rot_mat, trans_mat)

  img_2 = cv2.warpPerspective(
      img,
      transformation_matrix,
      (img.shape[1], img.shape[0]),
      borderMode=cv2.BORDER_WRAP
  )

  img_3 = cv2.bitwise_and(img_1, img_mask)
  img_4 = cv2.bitwise_and(img_2, np.invert(img_mask))
  return cv2.bitwise_or(img_3, img_4)

def apply_ripples(img, ripple_count, ripple_angle, phase):
  img_mask = np.zeros((width,height,3), np.uint8)

  ripple_radius = width / 2 / ripple_count

  for ripple in range(ripple_count, 0, -1):
    print (ripple_radius*ripple)
    if ripple%2 == 0:
      cv2.circle(img_mask, (width//2,height//2), round(ripple_radius*ripple), (255,255,255), thickness=-1)
    else:
      cv2.circle(img_mask, (width//2,height//2), round(ripple_radius*ripple), (0,0,0), thickness=-1)

  trans_mat = np.float32(
      [[1, 0, 0],
      [0, 1, 0]]
  )
  rot_mat = cv2.getRotationMatrix2D( (0,0), ripple_angle, 1 )
  trans_mat = np.vstack([trans_mat, [0,0,1]])
  rot_mat = np.vstack([rot_mat, [0,0,1]])
  transformation_matrix = np.matmul(rot_mat, trans_mat)

  img_1 = cv2.warpPerspective(
      img_0,
      transformation_matrix,
      (img_0.shape[1], img_0.shape[0]),
      borderMode=cv2.BORDER_WRAP
  )

  trans_mat = np.float32(
      [[1, 0, 0],
      [0, 1, 0]]
  )
  rot_mat = cv2.getRotationMatrix2D( (0,0), -ripple_angle, 1 )
  trans_mat = np.vstack([trans_mat, [0,0,1]])
  rot_mat = np.vstack([rot_mat, [0,0,1]])
  transformation_matrix = np.matmul(rot_mat, trans_mat)

  img_2 = cv2.warpPerspective(
      img_0,
      transformation_matrix,
      (img_0.shape[1], img_0.shape[0]),
      borderMode=cv2.BORDER_WRAP
  )

  img_3 = cv2.bitwise_and(img_1, img_mask)
  img_4 = cv2.bitwise_and(img_2, np.invert(img_mask))
  return cv2.bitwise_or(img_3, img_4)


def apply_pinch(img, angle):
  img_mask = np.zeros((400,400,3), np.uint8)
  img_mask[:,201:400] = (255,255,255)

  img_1 = apply_basic_transformation(img, (0,0), angle, 1, (0,0))
  img_2 = apply_basic_transformation(img, (0,0), -angle, 1, (0,0))

  return cv2.bitwise_or( 
                  cv2.bitwise_and(img_1, img_mask),
                  cv2.bitwise_and(img_2, np.invert(img_mask))
              )

def apply_explosion(img, translation):
  img_mask1 = np.zeros((400,400,3), np.uint8)
  img_mask2 = np.zeros((400,400,3), np.uint8)
  img_mask3 = np.zeros((400,400,3), np.uint8)
  img_mask4 = np.zeros((400,400,3), np.uint8)

  img_mask1[0:200,0:200] = (255,255,255)
  img_mask2[0:200,200:400] = (255,255,255)
  img_mask3[200:400,200:400] = (255,255,255)
  img_mask4[200:400,0:200] = (255,255,255)

  img_1 = apply_basic_transformation(img, (0,0), 0, 1, (-translation,-translation))
  img_2 = apply_basic_transformation(img, (0,0), 0, 1, (translation,-translation))
  img_3 = apply_basic_transformation(img, (0,0), 0, 1, (translation,translation))
  img_4 = apply_basic_transformation(img, (0,0), 0, 1, (-translation,translation))

  return cv2.bitwise_or( 
            cv2.bitwise_or( 
              cv2.bitwise_or( 
                  cv2.bitwise_and(img_1, img_mask1),
                  cv2.bitwise_and(img_2, img_mask2)),
              cv2.bitwise_and(img_3, img_mask3)),
            cv2.bitwise_and(img_4, img_mask4))

def apply_blur(img, blur_amt):
  return cv2.blur(img, (round(blur_amt),round(blur_amt)))

def apply_spiral(img, angle, ring_size, overlap, reverse, method):
  #angle = -50
  #ring_size = 50
  ring_count = math.ceil(283/ring_size)
  #overlap = 20
  #reverse = False
  #method = 1
  print(f'angle:{angle} ring_size:{ring_size}')

  img_final = np.zeros((400,400,3), np.uint8)

  for i in range(0,ring_count+1):
    img_mask = np.zeros((400,400,3), np.uint8)
    if method==1:
      img_mask = cv2.circle(img_mask, (200,200), i*ring_size, (255,255,255), ring_size+overlap)
    else:
      img_mask = cv2.circle(img_mask, (200,200), i*ring_size, (255,255,255), -1)
      if i>0:
        img_mask = cv2.circle(img_mask, (200,200), (i-1)*ring_size, (0,0,0), -1)
    #img_mask_list.append(img_mask)

    if reverse:
      img_ring = apply_basic_transformation(img, (0,0), (ring_count-i)*angle/ring_count, 1, (0,0))
    else:
      img_ring = apply_basic_transformation(img, (0,0), (i-1)*angle/ring_count, 1, (0,0))
    
    img_combine = cv2.bitwise_and(img_ring, img_mask)

    img_final = cv2.bitwise_or(img_final, img_combine)
    #cv2_imshow(img_final)

  return img_final

def apply_zoom(img, zoom, zoom_center):    
    # each point goes to center-zoom(center-pt)
    input_pts = np.float32([[0, 0], [width, 0], [width, height], [0, height]])
    output_pts = np.float32([[zoom_center[0]*(1-zoom[0]), zoom_center[1]*(1-zoom[1])], 
                             [zoom_center[0]-zoom[0]*(zoom_center[0]-width), zoom_center[1]*(1-zoom[1])], 
                             [zoom_center[0]-zoom[0]*(zoom_center[0]-width), zoom_center[1]-zoom[1]*(zoom_center[1]-height)], 
                             [zoom_center[0]*(1-zoom[0]), zoom_center[1]-zoom[1]*(zoom_center[1]-height)]])


    transform_mat = cv2.getPerspectiveTransform(input_pts, output_pts)
    M = cv2.getPerspectiveTransform(input_pts,output_pts)
    return cv2.warpPerspective(
          img,
          M,
          (width, height),
          flags=cv2.INTER_LINEAR,
          borderMode=cv2.BORDER_WRAP)
    
def apply_arrow(img, arrow_translate, isvertical):
  if isvertical:
    input_pts = np.float32([[0, 0], [200, 0], [200, 400], [0, 400]])
    output_pts = np.float32([[0, 0], [200, -arrow_translate], [200, 400-arrow_translate], [0, 400]])
  else:
    input_pts = np.float32([[0, 0], [400, 0], [400, 200], [0, 200]])
    output_pts = np.float32([[0, 0], [400, 0], [400+arrow_translate, 200], [arrow_translate, 200]])

  transform_mat = cv2.getPerspectiveTransform(input_pts, output_pts)
  M = cv2.getPerspectiveTransform(input_pts,output_pts)
  img_1 = cv2.warpPerspective(img_0,M,(400, 400),flags=cv2.INTER_LINEAR,borderMode=cv2.BORDER_WRAP)

  if isvertical:
    input_pts = np.float32([[200, 0], [400, 0], [400, 400], [200, 400]])
    output_pts = np.float32([[200, -arrow_translate], [400, 0], [400, 400], [200, 400-arrow_translate]])
  else:
    input_pts = np.float32([[0, 200], [400, 200], [400, 400], [0, 400]])
    output_pts = np.float32([[arrow_translate, 200], [400+arrow_translate, 200], [400, 400], [0, 400]])

  transform_mat = cv2.getPerspectiveTransform(input_pts, output_pts)
  M = cv2.getPerspectiveTransform(input_pts,output_pts)
  img_2 = cv2.warpPerspective(img_0,M,(400, 400),flags=cv2.INTER_LINEAR,borderMode=cv2.BORDER_WRAP)

  img_mask = np.zeros((400,400,3), np.uint8)  
  if isvertical:
    img_mask[:,0:200] = (255,255,255)
  else:
    img_mask[0:200,:] = (255,255,255)
    

  #cv2_imshow(cv2.bitwise_and(img_1, img_mask))
  #cv2_imshow(cv2.bitwise_and(img_2, np.invert(img_mask)))

  return cv2.bitwise_or(
        cv2.bitwise_and(img_1, img_mask), 
        cv2.bitwise_and(img_2, np.invert(img_mask))
  )

def apply_river(img, top_translate, bottom_translate):
  input_pts = np.float32([[0, 0], [400, 0], [400, 400], [0, 400]])
  output_pts = np.float32([[top_translate, 0], [400+top_translate, 0], [400+bottom_translate, 400], [bottom_translate, 400]])

  transform_mat = cv2.getPerspectiveTransform(input_pts, output_pts)
  M = cv2.getPerspectiveTransform(input_pts,output_pts)
  return cv2.warpPerspective(img_0,M,(400, 400),flags=cv2.INTER_LINEAR,borderMode=cv2.BORDER_WRAP)

def apply_windowshades(img, stripecount, phase, translation, isvertical):
  stripeheight=400/stripecount

  img_mask = np.zeros((400,400,3), np.uint8)  

  for stripe in range(0, math.ceil(stripecount/2)):
    top=round(stripe*stripeheight*2 + phase*stripeheight)
    bottom=round(top+stripeheight)
    if isvertical:
      img_mask[:,top:bottom] = (255,255,255)
    else:
      img_mask[top:bottom,:] = (255,255,255)

  if isvertical:
    trans_mat = np.float32(
        [[1, 0, 0],
        [0, 1, translation]]
    )
  else:
    trans_mat = np.float32(
        [[1, 0, translation],
        [0, 1, 0]]
    )
    
  rot_mat = cv2.getRotationMatrix2D( (0,0), 0, 1 )
  trans_mat = np.vstack([trans_mat, [0,0,1]])
  rot_mat = np.vstack([rot_mat, [0,0,1]])
  transformation_matrix = np.matmul(rot_mat, trans_mat)

  img_1 = cv2.warpPerspective(
      img,
      transformation_matrix,
      (img.shape[1], img.shape[0]),
      borderMode=cv2.BORDER_WRAP
  )

  if isvertical:
    trans_mat = np.float32(
        [[1, 0, 0],
        [0, 1, -translation]]
    )
  else:
    trans_mat = np.float32(
        [[1, 0, -translation],
        [0, 1, 0]]
    )
  trans_mat = np.vstack([trans_mat, [0,0,1]])
  transformation_matrix = np.matmul(rot_mat, trans_mat)

  img_2 = cv2.warpPerspective(
      img,
      transformation_matrix,
      (img.shape[1], img.shape[0]),
      borderMode=cv2.BORDER_WRAP
  )

  return cv2.bitwise_or(
        cv2.bitwise_and(img_1, img_mask), 
        cv2.bitwise_and(img_2, np.invert(img_mask))
  )

def apply_spin(img, spinX, spinY):
  # 1,1 is unchanged
  # -1,-1 is flipped on x and y
  # 0,0 is right in the middle (no pic)
  spin_amtY = 200 * (1-spinX)
  spin_amtX = 200 * (1-spinY)
  input_pts = np.float32([[0, 0], [400, 0], [400, 400], [0, 400]])
  output_pts = np.float32([[spin_amtX, spin_amtY], [400-spin_amtX, spin_amtY], [400-spin_amtX, 400-spin_amtY], [spin_amtX, 400-spin_amtY]])
  print (output_pts)

  transform_mat = cv2.getPerspectiveTransform(input_pts, output_pts)
  M = cv2.getPerspectiveTransform(input_pts,output_pts)
  return cv2.warpPerspective(img_0,M,(400, 400),flags=cv2.INTER_LINEAR,borderMode=cv2.BORDER_REFLECT101)

def apply_shuffle(img, count, translation):
  #count is the number of 4x4 squares across and down shuffling. 
  #   1 would be 4x4
  #   2 would be 8x8  etc

  #translation is the (x,y) amount to move square 1
  #   square 2 moves the same rotated 90  etc

  img_mask = []
  for i in range(0,4):
    img_mask.append(np.zeros((400,400,3), np.uint8))

  for x in range(0, count):
    x0 = round(x * (400/count))
    x2 = round((x+1) * (400/count)) 
    x1 = round((x0 + x2) / 2)

    for y in range(0, count):
      y0 = round(y * (400/count))
      y2 = round((y+1) * (400/count)) 
      y1 = round((y0 + y2) / 2)
      
      img_mask[0][x0:x1,y0:y1] = (255,255,255)
      img_mask[1][x1:x2,y0:y1] = (255,255,255)
      img_mask[2][x1:x2,y1:y2] = (255,255,255)
      img_mask[3][x0:x1,y1:y2] = (255,255,255)

  img_t = []

  img_t.append(apply_basic_transformation(img, (0,0), 0, 1, translation))
  img_t.append(apply_basic_transformation(img, (0,0), 0, 1, (-translation[1],-translation[0])))
  img_t.append(apply_basic_transformation(img, (0,0), 0, 1, (-translation[0],-translation[1])))
  img_t.append(apply_basic_transformation(img, (0,0), 0, 1, (translation[1],translation[0])))

  #spacer = np.zeros((400,5,3), np.uint8)
  #spacer[:,:] = (255,0,0)
  #out = cv2.hconcat([img_mask[0], spacer, img_mask[1], spacer, img_mask[2], spacer, img_mask[3]])
  #cv2_imshow(out)

  img_final = np.zeros((400,400,3), np.uint8)
  
  for i in range(0,4):
    img_combine = cv2.bitwise_and(img_t[i], img_mask[i])
    img_final = cv2.bitwise_or(img_final, img_combine)

  return img_final

from enum import Enum
class CornerCircleType(Enum):
  ALL_SAME = 1  # All corners rotate the same way
  OPPOSITES_SAME = 2  # Opposite corners rotate the same way; adjacent rotate the opposite way
  TOP_SAME = 3 # Top corners rotate the same way; bottom rotate the opposite way
  LEFT_SAME = 3 # Left corners rotate the same way; right rotate the opposite way

def apply_cornercircle(img, corner_angle, corner_zoom, corner_circle_type, center_angle, center_zoom):
  #corner_angle is the amount each corner circle rotates about the corner
  #  upper left rotates in that direction; the direction of the others are defined by type  
  #corner_zoom is the amount each corner circle zooms about the corner
  #corner_circle_type is one of these options from CornerCircleType enum
  #center_angle is the amount the center rotates about the center
  #center_zoom is the amount the center zooms from the center

  img_mask = []
  for i in range(0,5):
    img_mask.append(np.zeros((400,400,3), np.uint8))

  # get the corner masks
  img_mask[0] = cv2.circle(img_mask[0], (0,0), 200, (255,255,255), -1)
  img_mask[1] = cv2.circle(img_mask[1], (400,0), 200, (255,255,255), -1)
  img_mask[2] = cv2.circle(img_mask[2], (400,400), 200, (255,255,255), -1)
  img_mask[3] = cv2.circle(img_mask[3], (0,400), 200, (255,255,255), -1)

  # get the center mask
  for i in range(0,4):
    img_mask[4] = cv2.bitwise_or(img_mask[4], img_mask[i])
  img_mask[4] = cv2.bitwise_not(img_mask[4])

  img_t = []

  if corner_circle_type == CornerCircleType.ALL_SAME:
    angle_sign = [1,1,1,1]
  elif corner_circle_type == CornerCircleType.OPPOSITES_SAME:
    angle_sign = [1,-1,1,-1]
  elif corner_circle_type == CornerCircleType.TOP_SAME:
    angle_sign = [1,1,-1,-1]
  elif corner_circle_type == CornerCircleType.LEFT_SAME:
    angle_sign = [1,-1,-1,1]

  img_t.append(apply_basic_transformation(img, (-1,-1), angle_sign[0]*corner_angle, corner_zoom, (0,0)))
  img_t.append(apply_basic_transformation(img, (1,-1), angle_sign[1]*corner_angle, corner_zoom, (0,0)))
  img_t.append(apply_basic_transformation(img, (1,1), angle_sign[2]*corner_angle, corner_zoom, (0,0)))
  img_t.append(apply_basic_transformation(img, (-1,1), angle_sign[3]*corner_angle, corner_zoom, (0,0)))
  img_t.append(apply_basic_transformation(img, (0,0), center_angle, center_zoom, (0,0)))

  #cv2_imshow(cv2.hconcat([img_mask[0], spacer, img_mask[1], spacer, img_mask[2], spacer, img_mask[3], spacer, img_mask[4]]))

  img_final = np.zeros((400,400,3), np.uint8)
  
  for i in range(0,5):
    img_combine = cv2.bitwise_and(img_t[i], img_mask[i])
    img_final = cv2.bitwise_or(img_final, img_combine)

  return img_final

def apply_square_spiral(img, angle, square_size, reverse):
  square_count = 200//square_size
  img_final = np.zeros((400,400,3), np.uint8)

  for i in range(0,square_count+1):
    img_mask = np.zeros((400,400,3), np.uint8)
    start = square_size*i
    end = 400-start
    next_start = square_size*(i+1)
    next_end = 400 - next_start
    img_mask[start:end, start:end] = (255,255,255)
    img_mask[next_start:next_end, next_start:next_end] = (0,0,0)

    if reverse:
      img = apply_basic_transformation(img_0, (0,0), (square_count-i)*angle/square_count, 1, (0,0))
    else:
      img = apply_basic_transformation(img_0, (0,0), (i-1)*angle/square_count, 1, (0,0))
    
    img_combine = cv2.bitwise_and(img, img_mask)

    img_final = cv2.bitwise_or(img_final, img_combine)

  return img_final

def apply_waves(img, translate_x, count_x, phase_x, translate_y, count_y, phase_y):

  img_final = np.zeros((400,400,3), np.uint8)

  rows, cols = img.shape[:2]

  for i in range(rows): 
    for j in range(cols): 
      offset_x = round(translate_x * math.sin(2 * math.pi * (i / 400 * count_x + phase_x))) 
      offset_y = round(translate_y * math.sin(2 * math.pi * (j / 400 * count_y + phase_y))) 
      img_final[i,j] = img[(i+offset_y)%400,(j+offset_x)%400] 
            
  return img_final

def add_text(img, text, x, y, font, colorfill, coloroutline, outlinewidth):
    """If x or y is None, it will center
    """
    cv2.putText(img,  text, (50,250), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), 5, cv2.LINE_AA)


    return img

def merge_image(img1, img2, pct):
  print (pct, 1-pct)
  return cv2.addWeighted(img1,1-pct,img2,pct,0)



In [ ]:
#@title #**Fire up the A.I**
#@markdown #<a name="ai"></a>
 
import datetime
import time
from google.colab.patches import cv2_imshow
from IPython.display import clear_output 


# Delete memory from previous runs
!nvidia-smi -caa
for var in ['device', 'model', 'perceptor', 'z']:
  try:
    del globals()[var]
  except:
    pass

#try:
#    import gc
#    gc.collect()
#except:
#    pass

try:
    torch.cuda.empty_cache()
except:
    pass

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
if not key_frames:
    if text_prompts:
        print('Using text prompts:', text_prompts)
    if target_images:
        print('Using image prompts:', target_images)
if args.seed is None:
    seed = torch.seed()
else:
    seed = args.seed
torch.manual_seed(seed)
print('Using seed:', seed)
 
model = load_vqgan_model(args.vqgan_config, args.vqgan_checkpoint).to(device)
perceptor = clip.load(args.clip_model, jit=False)[0].eval().requires_grad_(False).to(device)
 
cut_size = perceptor.visual.input_resolution
e_dim = model.quantize.e_dim
f = 2**(model.decoder.num_resolutions - 1)
make_cutouts = MakeCutouts(cut_size, args.cutn, cut_pow=args.cut_pow)
n_toks = model.quantize.n_e
toksX, toksY = args.size[0] // f, args.size[1] // f
sideX, sideY = toksX * f, toksY * f
z_min = model.quantize.embedding.weight.min(dim=0).values[None, :, None, None]
z_max = model.quantize.embedding.weight.max(dim=0).values[None, :, None, None]
stop_on_next_loop = False  # Make sure GPU memory doesn't get corrupted from cancelling the run mid-way through, allow a full frame to complete

if restart_frame>0:
    restart_frame = restart_frame-1

start_time = time.time()
framecount = max_frames - restart_frame + 1

for i in range(restart_frame, max_frames):
    if stop_on_next_loop:
      break
    if key_frames:
        text_prompts = text_prompts_series[i]
        text_prompts = [phrase.strip() for phrase in text_prompts.split("|")]
        if text_prompts == ['']:
            text_prompts = []
        args.prompts = text_prompts

        target_images = target_images_series[i]

        if target_images == "None" or not target_images:
            target_images = []
        else:
            target_images = target_images.split("|")
            target_images = [image.strip() for image in target_images]
        args.image_prompts = target_images

        angle = angle_series[i]
        zoom = zoom_series[i]
        translation_x = translation_x_series[i]
        translation_y = translation_y_series[i]
        center_offset_x = center_offset_x_series[i]
        center_offset_y = center_offset_y_series[i]
        vtilt = vtilt_series[i]
        htilt = htilt_series[i]
        custom1 = custom1_series[i]
        custom2 = custom2_series[i]
        custom3 = custom3_series[i]
        custom4 = custom4_series[i]
        custom5 = custom5_series[i]
        iterations_per_frame = iterations_per_frame_series[i]
        print(
            f'text_prompts: {text_prompts}'
            f'angle: {angle}',
            f'zoom: {zoom}',
            f'translation_x: {translation_x}',
            f'translation_y: {translation_y}',
            f'center_offset_x: {center_offset_x}',
            f'center_offset_y: {center_offset_y}',
            f'vtilt: {vtilt}',
            f'htilt: {htilt}',
            f'custom1: {custom1}',
            f'custom2: {custom2}',
            f'custom3: {custom3}',
            f'custom4: {custom4}',
            f'custom5: {custom5}',
            f'iterations_per_frame: {iterations_per_frame}'
        )
    try:
        if i == 0 and initial_image != "":
            img_0 = read_image_workaround(initial_image)
            z, *_ = model.encode(TF.to_tensor(img_0).to(device).unsqueeze(0) * 2 - 1)
        elif i == 0 and not os.path.isfile(f'{working_dir}/steps/{i:05d}.png'):
            one_hot = F.one_hot(
                torch.randint(n_toks, [toksY * toksX], device=device), n_toks
            ).float()
            z = one_hot @ model.quantize.embedding.weight
            z = z.view([-1, toksY, toksX, e_dim]).permute(0, 3, 1, 2)
        else:
            if save_all_iterations:
                img_0 = read_image_workaround(
                    f'{working_dir}/steps/{i:05d}_{iterations_per_frame}.png')
            else:
                print(f'{working_dir}/steps/{i:05d}.png')
                img_0 = read_image_workaround(f'{working_dir}/steps/{i:05d}.png')


            pivot_x = int(np.floor(img_0.shape[0]*(.5+center_offset_x/2)))
            pivot_y = int(np.floor(img_0.shape[1]*(.5+center_offset_y/2)))
            center = (pivot_x, pivot_y)

            trans_mat = np.float32(
                [[1, 0, translation_x],
                [0, 1, translation_y]]
            )
            rot_mat = cv2.getRotationMatrix2D( center, angle, zoom )

            trans_mat = np.vstack([trans_mat, [0,0,1]])
            rot_mat = np.vstack([rot_mat, [0,0,1]])
            transformation_matrix = np.matmul(rot_mat, trans_mat)

            img_0 = cv2.warpPerspective(
                img_0,
                transformation_matrix,
                (img_0.shape[1], img_0.shape[0]),
                borderMode=cv2.BORDER_WRAP
            )

            if vtilt != 0:
              img_0 = apply_vtilt(img_0, vtilt)
              #img_0 = apply_pinch(img_0, vtilt)
            if htilt != 0:
              #img_0 = apply_htilt(img_0, htilt)
              img_0 = apply_explosion(img_0, htilt)

            #if i in range(19036,19224):
            #  a = (i-19036)/96*30+20
            #  if i>19128:
            #    a = 100-a
            #  img_0 = apply_spiral(img_0, -a, 1, 0, False, 2)
              #apply_spiral(img, angle, ring_size, overlap, reverse, method):

            if custom2 != 0:
              #img_0 = apply_spiral(img_0, custom, 1, 0, False, 2)
              #img_0 = apply_arrow(img_0, custom1, True)
              #img_0 = apply_waves(img_0, 0, 1, 0, custom1, custom2, 0)
              #img_0 = apply_windowshades(img_0, 2, custom2, custom1, True)

              if (custom1 > 0):
                nun_frame = round(custom1)%41
                print(f'/content/gdrive/MyDrive/vqgan/target/nun{nun_frame:02}.png')
                img_1 = read_image_workaround(f'/content/gdrive/MyDrive/vqgan/target/nun{nun_frame:02}.png')
                img_0 = merge_image(img_0, img_1, custom2)
              elif (custom3 > 0):
                nun_frame = round(custom3)%81
                print(f'/content/gdrive/MyDrive/vqgan/slomo/2x/{nun_frame:05}.png')
                img_1 = read_image_workaround(f'/content/gdrive/MyDrive/vqgan/slomo/2x/{nun_frame:05}.png')
                img_0 = merge_image(img_0, img_1, custom2)
              elif (custom4 > 0):
                nun_frame = round(custom4)%161
                print(f'/content/gdrive/MyDrive/vqgan/slomo/4x/{nun_frame:05}.png')
                img_1 = read_image_workaround(f'/content/gdrive/MyDrive/vqgan/slomo/4x/{nun_frame:05}.png')
                img_0 = merge_image(img_0, img_1, custom2)              

            #if custom2 != 0:
              #img_0 = apply_river(img_0, custom2, custom3)
              #img_0 = apply_windowshades(img_0, 8, 0, custom2, True)
            #  img_0 = apply_shuffle(img_0, 5, (custom2,0))
              

            #if custom3 != 0:
              #img_0 = apply_windowshades(img_0, 8, 0, custom3, False)
              #img_0 = apply_shuffle(img_0, 10, (custom3,0))
              #img_0 = apply_spiral(img_0, custom3, round(custom4), 0, False, 1)

            #if custom5 != 1:
              #img_0 = apply_shuffle(img_0, 20, (custom4,0))
              #img_0 = apply_spin(img_0, 1, custom5)
              # 1,1 is unchanged
              # -1,-1 is flipped on x and y
              # 0,0 is right in the middle (no pic)

            #if custom5 != 0:
              #img_0 = apply_spiral(img_0, custom5, 5, False, False, 1)
              #img_0 = apply_shuffle(img_0, 40, (custom5,0))


            z, *_ = model.encode(TF.to_tensor(img_0).to(device).unsqueeze(0) * 2 - 1)

        if i%100 == 0:
          clear_output()

        current_time = time.time()
        fraction_done =  (i-restart_frame+1) / (max_frames-restart_frame+1)
        est_total_time = (current_time-start_time)/fraction_done
        eta = start_time + est_total_time
        print (time.asctime(time.localtime(eta - 5*60*60)))

        i += 1

        z_orig = z.clone()
        z.requires_grad_(True)
        opt = optim.Adam([z], lr=args.step_size)

        normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                        std=[0.26862954, 0.26130258, 0.27577711])

        pMs = []

        for prompt in args.prompts:
            txt, weight, stop = parse_prompt(prompt)
            embed = perceptor.encode_text(clip.tokenize(txt).to(device)).float()
            pMs.append(Prompt(embed, weight, stop).to(device))

        for prompt in args.image_prompts:
            path, weight, stop = parse_prompt(prompt)
            img = resize_image(Image.open(path).convert('RGB'), (sideX, sideY))
            batch = make_cutouts(TF.to_tensor(img).unsqueeze(0).to(device))
            embed = perceptor.encode_image(normalize(batch)).float()
            pMs.append(Prompt(embed, weight, stop).to(device))

        for seed, weight in zip(args.noise_prompt_seeds, args.noise_prompt_weights):
            gen = torch.Generator().manual_seed(seed)
            embed = torch.empty([1, perceptor.visual.output_dim]).normal_(generator=gen)
            pMs.append(Prompt(embed, weight).to(device))

        def synth(z):
            z_q = vector_quantize(z.movedim(1, 3), model.quantize.embedding.weight).movedim(3, 1)
            return clamp_with_grad(model.decode(z_q).add(1).div(2), 0, 1)

        def add_xmp_data(filename):
            #imagen = ImgTag(filename=filename)
            #imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'creator', 'VQGAN+CLIP', {"prop_array_is_ordered":True, "prop_value_is_array":True})
            #if args.prompts:
            #    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'title', " | ".join(args.prompts), {"prop_array_is_ordered":True, "prop_value_is_array":True})
            #else:
            #    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'title', 'None', {"prop_array_is_ordered":True, "prop_value_is_array":True})
            #imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'i', str(i), {"prop_array_is_ordered":True, "prop_value_is_array":True})
            #imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'model', model_name, {"prop_array_is_ordered":True, "prop_value_is_array":True})
            #imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'seed',str(seed) , {"prop_array_is_ordered":True, "prop_value_is_array":True})
            #imagen.close()
            xxxxxx=1

        def add_stegano_data(filename):
            data = {
                "title": " | ".join(args.prompts) if args.prompts else None,
                "notebook": "VQGAN+CLIP",
                "i": i,
                "model": model_name,
                "seed": str(seed),
            }
            lsb.hide(filename, json.dumps(data)).save(filename)

        @torch.no_grad()
        def checkin(i, losses):
            losses_str = ', '.join(f'{loss.item():g}' for loss in losses)
            tqdm.write(f'i: {i}, loss: {sum(losses).item():g}, losses: {losses_str}')
            out = synth(z)
            TF.to_pil_image(out[0].cpu()).save('progress.png')
            add_stegano_data('progress.png')
            add_xmp_data('progress.png')
            display.display(display.Image('progress.png'))

        def save_output(i, img, suffix=None):
            filename = \
                f"{working_dir}/steps/{i:05}{'_' + suffix if suffix else ''}.png"
            imageio.imwrite(filename, np.array(img))
            add_stegano_data(filename)
            add_xmp_data(filename)

        def save_with_message(i, message, x, y, font, colorfill, coloroutline, outlinewidth):
            """If x or y is None, it will center
            """            
            filename = \
                f"{working_dir}/steps/{i:05}.png"
            my_image = Image.open(filename)
            image_editable = ImageDraw.Draw(my_image)

            if x is None or y is None:
              textwidth, textheight = image_editable.textsize(message, font=font)
              if x is None:
                x = (width - textwidth)/2
              if y is None:
                y = (height - textheight)/2              

            image_editable.text((x,y), message, fill=colorfill, font=font, 
                                stroke_width=outlinewidth, stroke_fill=coloroutline)
            my_image.save(filename)
            add_stegano_data(filename)
            add_xmp_data(filename)

        def save_with_overlay(i, overlayfilename, x, y):
            filename = \
                f"{working_dir}/steps/{i:05}.png"
            my_image = Image.open(filename)
            my_overlay = Image.open(overlayfilename).convert("RGBA")

            my_image.paste(my_overlay, (x, y), my_overlay)

            my_image.save(filename)
            add_stegano_data(filename)
            add_xmp_data(filename)

        def save_with_blend(i, overlayfilename, blendamt):
            filename = \
                f"{working_dir}/steps/{i:05}.png"
            my_image = Image.open(filename)
            my_overlay = Image.open(overlayfilename).convert("RGBA")

            blended = Image.blend(my_image, my_overlay, alpha=blendamt)

            blended.save(filename)
            add_stegano_data(filename)
            add_xmp_data(filename)

        def ascend_txt(i, j, save=True, suffix=None):
            out = synth(z)
            iii = perceptor.encode_image(normalize(make_cutouts(out))).float()

            result = []

            if args.init_weight:
                result.append(F.mse_loss(z, z_orig) * args.init_weight / 2)

            for prompt in pMs:
                result.append(prompt(iii))
            img = np.array(out.mul(255).clamp(0, 255)[0].cpu().detach().numpy().astype(np.uint8))[:,:,:]
            img = np.transpose(img, (1, 2, 0))

            if save:

               if custom5 != 0:
                  if (custom1 > 0):
                    nun_frame = round(custom1)%41
                    print(f'/content/gdrive/MyDrive/vqgan/target/nun{nun_frame:02}.png')
                    img_1 = read_image_workaround(f'/content/gdrive/MyDrive/vqgan/target/nun{nun_frame:02}.png')
                    img = merge_image(img, img_1, custom5)
                  elif (custom3 > 0):
                    nun_frame = round(custom3)%81
                    print(f'/content/gdrive/MyDrive/vqgan/slomo/2x/{nun_frame:05}.png')
                    img_1 = read_image_workaround(f'/content/gdrive/MyDrive/vqgan/slomo/2x/{nun_frame:05}.png')
                    img = merge_image(img, img_1, custom5)
                  elif (custom4 > 0):
                    nun_frame = round(custom4)%161
                    print(f'/content/gdrive/MyDrive/vqgan/slomo/4x/{nun_frame:05}.png')
                    img_1 = read_image_workaround(f'/content/gdrive/MyDrive/vqgan/slomo/4x/{nun_frame:05}.png')
                    img = merge_image(img, img_1, custom5)              

               save_output(i, img, suffix=suffix)

                #if i>45500 and i%12==0:
                #  font_size = 64
                #  title_font = ImageFont.truetype('/content/gdrive/MyDrive/fonts/flow.ttf', size=font_size)
                #  title_text = "Hubajube"
                #  save_with_message(i, title_text, None, None, title_font, 'blue', 'black', 2)

                #if i>=681 and i<=692:
                #  font_size = 64
                #  title_font = ImageFont.truetype('/content/gdrive/MyDrive/fonts/OstrichSans-Heavy.otf', size=font_size)
                #  if i%2 == 0:
                #    title_text = "Un"
                #    save_with_message(i, title_text, 83.5, 120, title_font, 'white', 'black', 2)
                #  else:
                #    title_text = "natural"
                #    save_with_message(i, title_text, 135, 120, title_font, 'white', 'black', 2)

                #if i in range(8491, 8592):
                #  if i%6 == 1:
                #    save_with_overlay(i, '/content/gdrive/MyDrive/vqgan/tests/spark3.png', 0, 0)


            return result

        def train(i, j, save=True, suffix=None):
            opt.zero_grad()
            lossAll = ascend_txt(i, j, save=save, suffix=suffix)

            if i % args.display_freq == 0 and save:
                checkin(i, lossAll)

            #losses_str = ', '.join(f'{loss.item():g}' for loss in lossAll)
            #tqdm.write(f'i: {i}, loss: {sum(lossAll).item():g}, lossAll: {losses_str}')

            loss = sum(lossAll)
            loss.backward()
            opt.step()
            with torch.no_grad():
                z.copy_(z.maximum(z_min).minimum(z_max))

        with tqdm() as pbar:
            if iterations_per_frame == 0:
                save_output(i, img_0)
            j = 1
            while True:
                suffix = (str(j) if save_all_iterations else None)
                if j >= iterations_per_frame:
                    train(i, j, save=True, suffix=suffix)
                    break
                if save_all_iterations:
                    train(i, j, save=True, suffix=suffix)
                else:
                    train(i, j, save=False, suffix=suffix)
                j += 1
                pbar.update()
    except KeyboardInterrupt:
      stop_on_next_loop = True
      pass

Wed Feb  2 18:19:19 2022


0it [00:00, ?it/s]

text_prompts: ['coins: 0.0', 'sun: 5.0', 'cartoon: 0.05']angle: 4.0 zoom: 1.05 translation_x: 0.0 translation_y: 0.0 center_offset_x: 0.0 center_offset_y: 0.0 vtilt: 0.0 htilt: 0.0 custom1: 0.0 custom2: 0.0 custom3: 0.0 custom4: 0.0 custom5: 0.0 iterations_per_frame: 15
/content/gdrive/MyDrive/vqgan/steps/00101.png
/content/gdrive/MyDrive/vqgan/steps/00101.png
Wed Feb  2 18:19:19 2022


0it [00:00, ?it/s]

text_prompts: ['coins: 0.0', 'sun: 5.0', 'cartoon: 0.1']angle: 4.0 zoom: 1.05 translation_x: 0.0 translation_y: 0.0 center_offset_x: 0.0 center_offset_y: 0.0 vtilt: 0.0 htilt: 0.0 custom1: 0.0 custom2: 0.0 custom3: 0.0 custom4: 0.0 custom5: 0.0 iterations_per_frame: 15
/content/gdrive/MyDrive/vqgan/steps/00102.png
/content/gdrive/MyDrive/vqgan/steps/00102.png
Wed Feb  2 18:19:18 2022


...

text_prompts: ['coins: 0.0', 'sun: 5.0', 'cartoon: 0.35000000000000003']angle: 4.0 zoom: 1.05 translation_x: 0.0 translation_y: 0.0 center_offset_x: 0.0 center_offset_y: 0.0 vtilt: 0.0 htilt: 0.0 custom1: 0.0 custom2: 0.0 custom3: 0.0 custom4: 0.0 custom5: 0.0 iterations_per_frame: 15
/content/gdrive/MyDrive/vqgan/steps/00107.png
/content/gdrive/MyDrive/vqgan/steps/00107.png
Wed Feb  2 18:19:16 2022


0it [00:00, ?it/s]

text_prompts: ['coins: 0.0', 'sun: 5.0', 'cartoon: 0.4']angle: 4.0 zoom: 1.05 translation_x: 0.0 translation_y: 0.0 center_offset_x: 0.0 center_offset_y: 0.0 vtilt: 0.0 htilt: 0.0 custom1: 0.0 custom2: 0.0 custom3: 0.0 custom4: 0.0 custom5: 0.0 iterations_per_frame: 15
/content/gdrive/MyDrive/vqgan/steps/00108.png
/content/gdrive/MyDrive/vqgan/steps/00108.png
Wed Feb  2 18:19:16 2022


0it [00:00, ?it/s]

text_prompts: ['coins: 0.0', 'sun: 5.0', 'cartoon: 0.45']angle: 4.0 zoom: 1.05 translation_x: 0.0 translation_y: 0.0 center_offset_x: 0.0 center_offset_y: 0.0 vtilt: 0.0 htilt: 0.0 custom1: 0.0 custom2: 0.0 custom3: 0.0 custom4: 0.0 custom5: 0.0 iterations_per_frame: 15
/content/gdrive/MyDrive/vqgan/steps/00109.png
/content/gdrive/MyDrive/vqgan/steps/00109.png


# **Optional: SRCNN for increasing resolution**

In [ ]:
#@markdown ## **Install SRCNN** 
srcnn = False#@param {type:"boolean"} 

if srcnn:
  import shutil

  !git clone https://github.com/Mirwaisse/SRCNN.git
  !curl https://raw.githubusercontent.com/justinjohn0306/SRCNN/master/models/model_2x.pth -o model_2x.pth
  !curl https://raw.githubusercontent.com/justinjohn0306/SRCNN/master/models/model_3x.pth -o model_3x.pth
  !curl https://raw.githubusercontent.com/justinjohn0306/SRCNN/master/models/model_4x.pth -o model_4x.pth

In [ ]:
#@markdown ## **Increase Resolution**


if srcnn:
  # import subprocess in case this cell is run without the above cells
  import subprocess
  # Set zoomed = True if this cell is run
  zoomed = True

  init_frame = 45933#@param {type:"number"}
  last_frame = 46100#@param {type:"number"}
  zoomsize = "3" #@param ["2", "3", "4"]
  restart_frame = 0
  working_dir='/content/gdrive/MyDrive/vqgan'

  modelpath = '/content/model_' + zoomsize + 'x.pth'

  if restart_frame>0:
    init_frame = restart_frame

  if init_frame < 1:
    init_frame = 1

  for i in range(init_frame, last_frame+1): #
      filename = f"{i:05}.png"
      cmd = [
          'python',
          '/content/SRCNN/run.py',
          '--zoom_factor',
          zoomsize,  # Note if you increase this, you also need to change the model.
          '--model',
          modelpath,  # 2x, 3x and 4x are available from the repo above
          '--image',
          filename,
          '--cuda'
      ]
      print(f'Upscaling {filename}')

      process = subprocess.Popen(cmd, cwd=f'{working_dir}/steps/')
      stdout, stderr = process.communicate()
      if process.returncode != 0:
          print(cmd)
          print(working_dir)
          print(stderr)
          raise RuntimeError(stderr)
      else:
        shutil.move(f'{working_dir}/steps/zoomed_{filename}', f'{working_dir}/zoomed/zoomed_{filename}')



## **Make a video of the results**

To generate a video with the frames, run the cell below. You can modify the number of FPS, the initial frame, the last frame, etc.

In [ ]:
# @title ### **Create video**

makevideo = True#@param {type:"boolean"} 

if makevideo:
  # import subprocess in case this cell is run without the above cells
  import subprocess

  filepath='/content/gdrive/MyDrive/vqgan/video.mp4'
  working_dir='/content/gdrive/MyDrive/vqgan'
  restart_frame=0

  zoomed = True#@param {type:"boolean"} 
  init_frame = 0#@param {type:"number"} This is the frame where the video will start
  last_frame = 200#@param {type:"number"} You can change i to the number of the last frame you want to generate. It will raise an error if that number of frames does not exist.
  fps = 12#@param {type:"number"}

  #frames = []
  frame = last_frame - init_frame + 1
  # tqdm.write('Generating video...')
  #try:
  #    zoomed
  #except NameError:
  #    image_path = f'{working_dir}/steps/%04d.png'
  #else:
  #    image_path = f'{working_dir}/steps/zoomed_%05d.png'
  if zoomed:
    image_path = f'{working_dir}/zoomed/zoomed_%05d.png'
  else:
    image_path = f'{working_dir}/steps/%05d.png'

  print(image_path)

  cmd = [
      'ffmpeg',
      '-y',
      '-vcodec',
      'png',
      '-r',
      str(fps),
      '-start_number',
      str(init_frame),
      '-i',
      image_path,
      '-frames:v',
      str(frame),
      '-c:v',
      'libx264',
      '-vf',
      f'fps={fps}',
      '-pix_fmt',
      'yuv420p',
      '-crf',
      '17',
      '-preset',
      'veryslow',
      filepath
  ]

  process = subprocess.Popen(cmd, cwd=f'{working_dir}/steps/', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  stdout, stderr = process.communicate()
  if process.returncode != 0:
      print(stderr)
      print(
          "You may be able to avoid this error by backing up the frames,"
          "restarting the notebook, and running only the video synthesis cells,"
          "or by decreasing the resolution of the image generation steps. "
          "If you restart the notebook, you will have to define the `filepath` manually"
          "by adding `filepath = 'PATH_TO_THE_VIDEO'` to the beginning of this cell. "
          "If these steps do not work, please post the traceback in the github."
      )
      raise RuntimeError(stderr)
  else:
      print("The video is ready")

In [ ]:
# @title **See video in the browser**
# @markdown This process may take a little longer. If you don't want to wait, download it by executing the next cell instead of using this cell.
#mp4 = open(filepath,'rb').read()
#data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
#display.HTML("""
#<video width=400 controls>
#      <source src="%s" type="video/mp4">
#</video>
#""" % data_url)

In [ ]:
# @title **Download video**
#from google.colab import files
#files.download(filepath)

# Optional: Super-Slomo for smoothing movement

This step might run out of memory if you run it right after the steps above. If it does, restart the notebook, upload a saved copy of the video from the previous step (or get it from google drive) and define the variable `filepath` with the path to the video before running the cells below again

In [ ]:
# @title **Download Super-Slomo model**
slomo = False#@param {type:"boolean"} 

if slomo:

  !git clone -q --depth 1 https://github.com/avinashpaliwal/Super-SloMo.git
  from os.path import exists
  def download_from_google_drive(file_id, file_name):
    # download a file from the Google Drive link
    !rm -f ./cookie
    !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id={file_id}" > /dev/null
    confirm_text = !awk '/download/ {print $NF}' ./cookie
    confirm_text = confirm_text[0]
    !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm={confirm_text}&id={file_id}" -o {file_name}
    
  pretrained_model = 'SuperSloMo.ckpt'
  if not exists(pretrained_model):
    download_from_google_drive('1IvobLDbRiBgZr3ryCRrWL8xDbMZ-KnpF', pretrained_model)

In [ ]:
# import subprocess in case this cell is run without the above cells

if slomo:

  import subprocess

  SLOW_MOTION_FACTOR = 2#@param {type:"number"}
  TARGET_FPS = 12#@param {type:"number"}
  filepath='/content/gdrive/MyDrive/vqgan/video.mp4'

  if os.path.exists(f'{filepath}-slomo.mkv'):
    print(f'Deleting {filepath}-slomo.mkv')
    os.remove(f'{filepath}-slomo.mkv')

  if os.path.exists(f'{filepath}-slomo.mp4'):
    print(f'Deleting {filepath}-slomo.mp4')
    os.remove(f'{filepath}-slomo.mp4')

  cmd1 = [
      'python',
      'Super-SloMo/video_to_slomo.py',
      '--checkpoint',
      pretrained_model,
      '--video',
      filepath,
      '--sf',
      str(SLOW_MOTION_FACTOR),
      '--fps',
      str(TARGET_FPS),
      '--output',
      f'{filepath}-slomo.mkv',
  ]
  process = subprocess.Popen(cmd1, cwd=f'/content', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  stdout, stderr = process.communicate()
  if process.returncode != 0:
      print(stderr)
      raise RuntimeError(stderr)

  cmd2 = [
      'ffmpeg',
      '-i',
      f'{filepath}-slomo.mkv',
      '-pix_fmt',
      'yuv420p',
      '-crf',
      '17',
      '-preset',
      'veryslow',
      f'{filepath}-slomo.mp4',
  ]

  process = subprocess.Popen(cmd2, cwd=f'/content', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  stdout, stderr = process.communicate()
  if process.returncode != 0:
      print(stderr)
      raise RuntimeError(stderr)


In [ ]:
# @title **See video in the browser**
# @markdown This process may take a little longer. If you don't want to wait, download it by executing the next cell instead of using this cell.
#from base64 import b64encode
#from IPython import display
#mp4 = open(f'{filepath}-slomo.mp4','rb').read()
#data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
#display.HTML("""
#<video width=400 controls>
#      <source src="%s" type="video/mp4">
#</video>
#""" % data_url)

In [ ]:
# @title **Download video**
#from google.colab import files
#files.download(f'{filepath}-slomo.mp4')

JS to prevent idle timeout:

Press F12 OR CTRL + SHIFT + I OR right click on this website -> inspect.
Then click on the console tab and paste in the following code.

```javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
```